In [1]:
import pickle
from sympy import *
from binarize_features import *

language_model = 'bert-large-cased'
country_file = 'data/country_list_continents.csv'
occ_file = 'data/occupations_subset.csv'
binarizer = Binarizer(country_file, 5, occ_file)

### Loading

Load the extracted rule set and the background used from the corresponding files.

In [2]:
with open('data/' + language_model + '_rules.txt', 'rb') as f:
    h = pickle.load(f)
with open('data/background.txt', 'rb') as f:
    background = pickle.load(f)

In [3]:
gender_related = []
all_rules = []
for rule in h:
    if rule not in background:
        expr = sstr(rule)
        all_rules.append(expr)
        print(expr)
        if 'v27' in expr:
            gender_related.append(expr)

Implies(v23 & v27 & v3 & v8, v0)
Implies(v14 & v17 & v2 & v27, v12)
Implies(v14 & v17 & v2 & v27, v10)
Implies(v16 & v18 & v27 & v3, v1)
Implies(v16 & v18 & v27 & v3, v11)
Implies(v14 & v17 & v2 & v27, v7)
Implies(v14 & v17 & v2 & v27, v13)
Implies(v16 & v18 & v27 & v3, v2)
Implies(v16 & v18 & v27 & v3, v4)
Implies(v16 & v18 & v27 & v3, v22)
Implies(v16 & v18 & v27 & v3, v5)
Implies(v16 & v18 & v27 & v3, v6)
Implies(v0, v13)
Implies(v14 & v17 & v2 & v27, v8)
Implies(v23 & v27 & v3 & v8, v20)
Implies(v23 & v27 & v3 & v8, v25)
~(v16 & v18 & v27 & v3)
Implies(v23 & v27 & v3 & v8, v26)
Implies(v23 & v27 & v3 & v8, v21)
Implies(v23 & v27 & v3 & v8, v15)
Implies(v14 & v17 & v2 & v27, v9)
Implies(v23 & v27 & v3 & v8, v17)
Implies(v16 & v18 & v27 & v3, v0)
Implies(v23 & v27 & v3 & v8, v12)
Implies(v14 & v17 & v2 & v27, v19)
Implies(v23 & v27 & v3 & v8, v10)
Implies(v23 & v27 & v3 & v8, v13)
Implies(v23 & v27 & v3 & v8, v7)
~(v23 & v27 & v3 & v8)
Implies(v14 & v17 & v2 & v27, v24)
Implies(v14 &

In [8]:
rules = []
for rule in all_rules:
    if '~' in rule:
        idx = []
        if not '(' in rule:
            idx.append(int(rule.replace('~v', '')))
        else:
            for s in rule.split(' '):
                if 'v' in s:
                    s = s.replace('~(', '')
                    s = s.replace(')', '')
                    idx.append(int(s.replace('v', '')))
            rules.append({'head' : '','body' : idx})
    else:
        body, head = rule.split(', ')
        body_idx = []
        for s in body.split(' '):
            if 'v' in s:
                s = s.replace('Implies(', '')
                body_idx.append(int(s.replace('v', '')))
        head_idx = head.replace('v', '').replace(')','')
        rules.append({'head' : head_idx, 'body' : body_idx})
print(rules)

[{'head': '0', 'body': [23, 27, 3, 8]}, {'head': '12', 'body': [14, 17, 2, 27]}, {'head': '10', 'body': [14, 17, 2, 27]}, {'head': '1', 'body': [16, 18, 27, 3]}, {'head': '11', 'body': [16, 18, 27, 3]}, {'head': '7', 'body': [14, 17, 2, 27]}, {'head': '13', 'body': [14, 17, 2, 27]}, {'head': '2', 'body': [16, 18, 27, 3]}, {'head': '4', 'body': [16, 18, 27, 3]}, {'head': '22', 'body': [16, 18, 27, 3]}, {'head': '5', 'body': [16, 18, 27, 3]}, {'head': '6', 'body': [16, 18, 27, 3]}, {'head': '13', 'body': [0]}, {'head': '8', 'body': [14, 17, 2, 27]}, {'head': '20', 'body': [23, 27, 3, 8]}, {'head': '25', 'body': [23, 27, 3, 8]}, {'head': '', 'body': [16, 18, 27, 3]}, {'head': '26', 'body': [23, 27, 3, 8]}, {'head': '21', 'body': [23, 27, 3, 8]}, {'head': '15', 'body': [23, 27, 3, 8]}, {'head': '9', 'body': [14, 17, 2, 27]}, {'head': '17', 'body': [23, 27, 3, 8]}, {'head': '0', 'body': [16, 18, 27, 3]}, {'head': '12', 'body': [23, 27, 3, 8]}, {'head': '19', 'body': [14, 17, 2, 27]}, {'head

In [9]:
total_lookup = []
for index in range(len(binarizer.age_containers) + 1):
    if index == 0:
        total_lookup.append("before " + str(binarizer.age_containers[index]))
    elif index == len(binarizer.age_containers):
        total_lookup.append("after " + str(binarizer.age_containers[index-1]))
    else:
        total_lookup.append("between " + str(binarizer.age_containers[index-1]) + " and " + str(binarizer.age_containers[index]))
for v in binarizer.continent_lookup.keys():
    total_lookup.append(v)
for v in binarizer.occupation_lookup.keys():
    total_lookup.append(v)
total_lookup.append('male')
print({i : total_lookup[i] for i in range(len(total_lookup))})

{0: 'before 1892', 1: 'between 1892 and 1934', 2: 'between 1934 and 1956', 3: 'between 1956 and 1976', 4: 'after 1976', 5: 'North America', 6: 'Africa', 7: 'Europe', 8: 'Asia', 9: 'South America', 10: 'Insular Oceania', 11: 'Eurasia', 12: 'Central America', 13: '?', 14: 'Oceania', 15: 'Australia', 16: 'Americas', 17: 'fashion_designer', 18: 'nurse', 19: 'dancer', 20: 'priest', 21: 'footballer', 22: 'banker', 23: 'singer', 24: 'lawyer', 25: 'mathematician', 26: 'diplomat', 27: 'male'}


In [10]:
string_rules = []
for rule in rules:
    s = ""
    for variable in rule['body']:
        s = s + total_lookup[variable] + " & "
    s = s[:-2]
    if rule['head'] != '':
        s = s + " ---> " + total_lookup[int(rule['head'])]
    #if 'fashion_designer' not in s:
    string_rules.append(s)

In [11]:
string_rules.sort()
for s in string_rules:
    print(s)

Americas & nurse & male & between 1956 and 1976 
Americas & nurse & male & between 1956 and 1976  ---> ?
Americas & nurse & male & between 1956 and 1976  ---> Africa
Americas & nurse & male & between 1956 and 1976  ---> Asia
Americas & nurse & male & between 1956 and 1976  ---> Australia
Americas & nurse & male & between 1956 and 1976  ---> Central America
Americas & nurse & male & between 1956 and 1976  ---> Eurasia
Americas & nurse & male & between 1956 and 1976  ---> Europe
Americas & nurse & male & between 1956 and 1976  ---> Insular Oceania
Americas & nurse & male & between 1956 and 1976  ---> North America
Americas & nurse & male & between 1956 and 1976  ---> Oceania
Americas & nurse & male & between 1956 and 1976  ---> South America
Americas & nurse & male & between 1956 and 1976  ---> after 1976
Americas & nurse & male & between 1956 and 1976  ---> banker
Americas & nurse & male & between 1956 and 1976  ---> before 1892
Americas & nurse & male & between 1956 and 1976  ---> betw